In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

import models 
import utils

import math
from functools import partial

import torch
from torch import nn
import torch.nn.utils.parametrize as parametrize
import lora 
# import utils


import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
import numpy as np


import torch
import torch.nn as nn
import torch.nn.utils.parametrize as parametrize

In [2]:
device = 'cuda'

mean_cifar10 = (0.4914, 0.4822, 0.4465)
std_cifar10 = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10)
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)


Files already downloaded and verified


In [4]:
model = models.resnet.resnet50(pretrained=True, progress=True, device="cuda").to('cuda')

layers = [ model.conv1, model.layer1[0].conv1, model.layer1[0].conv2, model.layer1[0].conv3,
                model.layer2[0].conv1, model.layer2[0].conv2, model.layer2[0].conv3,
                model.layer3[0].conv1, model.layer3[0].conv2, model.layer3[0].conv3,
                model.layer4[0].conv1, model.layer4[0].conv2, model.layer4[0].conv3, model.fc ]

for conv_layer in layers:
    lora_param = lora.layer_parametrization(conv_layer, device="cuda", rank=10, lora_alpha=1)
    parametrize.register_parametrization(conv_layer, 'weight', lora_param)

torch.Size([64, 3, 3, 3]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([64, 64, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([64, 64, 3, 3]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([256, 64, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([128, 256, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([128, 128, 3, 3]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([512, 128, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([256, 512, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([256, 256, 3, 3]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([1024, 256, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([512, 1024, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([512, 512, 3, 3]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([2048, 512, 1, 1]) <class 'torch.nn.modules.conv.Conv2d'>
torch.Size([10, 2048]) <class 'torch.nn.modules.linear.Linear'>


In [4]:
for n,p in model.named_parameters():
    print(n)

conv1.parametrizations.weight.original
conv1.parametrizations.weight.0.mat_A
conv1.parametrizations.weight.0.mat_B
bn1.weight
bn1.bias
layer1.0.conv1.parametrizations.weight.original
layer1.0.conv1.parametrizations.weight.0.mat_A
layer1.0.conv1.parametrizations.weight.0.mat_B
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.parametrizations.weight.original
layer1.0.conv2.parametrizations.weight.0.mat_A
layer1.0.conv2.parametrizations.weight.0.mat_B
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.parametrizations.weight.original
layer1.0.conv3.parametrizations.weight.0.mat_A
layer1.0.conv3.parametrizations.weight.0.mat_B
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias


In [5]:
for name, param in model.named_parameters():
    if 'mat' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

for layer in layers:
  layer.parametrizations["weight"][0].requires_grad = True

for name, param in model.named_parameters():
   if param.requires_grad:
        print(f"Parameter: {name}, Shape: {param.size()}")


Freezing non-LoRA parameter conv1.parametrizations.weight.original
Freezing non-LoRA parameter bn1.weight
Freezing non-LoRA parameter bn1.bias
Freezing non-LoRA parameter layer1.0.conv1.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn1.weight
Freezing non-LoRA parameter layer1.0.bn1.bias
Freezing non-LoRA parameter layer1.0.conv2.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn2.weight
Freezing non-LoRA parameter layer1.0.bn2.bias
Freezing non-LoRA parameter layer1.0.conv3.parametrizations.weight.original
Freezing non-LoRA parameter layer1.0.bn3.weight
Freezing non-LoRA parameter layer1.0.bn3.bias
Freezing non-LoRA parameter layer1.0.downsample.0.weight
Freezing non-LoRA parameter layer1.0.downsample.1.weight
Freezing non-LoRA parameter layer1.0.downsample.1.bias
Freezing non-LoRA parameter layer1.1.conv1.weight
Freezing non-LoRA parameter layer1.1.bn1.weight
Freezing non-LoRA parameter layer1.1.bn1.bias
Freezing non-LoRA parameter layer1

In [7]:
# Setup device
import torch.optim as optim
import active

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean_cifar10 = (0.4914, 0.4822, 0.4465)
std_cifar10 = (0.2471, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10)  ])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
pool_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
pool_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize model, optimizer, and loss function
model = models.resnet.resnet50(pretrained=True, progress=True, device="cuda").to('cuda')
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

adapt_dataset = active.EmptyDataset()

pool_indices = list( range(len(pool_loader.dataset ) ) ) 
n_queries = 6
active_batch_size = 10

import trades

for i in range(n_queries):

    print( 'iteration {}'.format(i) )
    
    pool_loader = DataLoader( Subset(pool_dataset, pool_indices), batch_size=128, shuffle=False)
    query_indices = active.uncertainty_sampling(model, pool_loader, active_batch_size)

    global_query_indices = [ pool_indices[idx] for idx in query_indices]

    for idx in global_query_indices:
        image, label = pool_dataset[idx]
        adapt_dataset.add_data( [image], [label] )

    # Remove queried instances from the pool
    pool_indices = [idx for idx in pool_indices if idx not in global_query_indices]
    print( len(pool_indices) )

    ################# Update the ResNet through low rank matrices:
    print('start training process')

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    adapt_loader = DataLoader(adapt_dataset, batch_size=64, shuffle=True)
    for _ in range(10):
        print('epoch {}'.format(_) )
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = trades.trades_loss(model=model, x_natural=data, y=target, optimizer=optimizer,)
            print(loss)
            loss.backward()
            optimizer.step()


Files already downloaded and verified
iteration 0
9990
start training process
epoch 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 2.94 GiB of which 22.12 MiB is free. Including non-PyTorch memory, this process has 2.91 GiB memory in use. Of the allocated memory 2.78 GiB is allocated by PyTorch, and 25.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [20]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
launch_experiment(model, device, train_loader, test_loader, optimizer, 20, epoch_AT_vanilla)

Train Err	Test Err	Adv Err


  0%|          | 0/20 [00:00<?, ?it/s]

0.112550	0.020900	0.040600
0.039983	0.016300	0.033000
0.030117	0.011900	0.030900
0.025800	0.012600	0.025900
0.023600	0.009500	0.022500
0.019617	0.008200	0.022300
0.018083	0.010200	0.021000
0.016300	0.010500	0.021500
0.015217	0.006700	0.020500
0.013550	0.008700	0.021400
0.012383	0.009200	0.020500
0.012433	0.008500	0.022100
0.010983	0.007900	0.020000
0.009767	0.006800	0.019400
0.009983	0.006900	0.019200
0.008517	0.008600	0.021000
0.008683	0.007400	0.019600
0.008083	0.006200	0.017900
0.007750	0.007200	0.018800
0.007033	0.005700	0.017000


In [ ]:
# torch.save(model.state_dict(), './models/lora_robust_model.pth')
# model.load_state_dict(torch.load('./models/lora_robust_model.pth'))

In [11]:
print( model.fc1.parametrizations.weight.original )
print( model.fc1.parametrizations.weight[0].mat_A )
print( model.fc1.parametrizations.weight[0].mat_B )

Parameter containing:
tensor([[ 0.4101,  0.6101, -0.7784,  ...,  0.1910, -0.3317,  0.3116],
        [ 0.2397, -0.1602, -0.0614,  ..., -0.0139,  0.7553, -0.2833],
        [-0.2825, -0.0330, -0.0121,  ...,  0.6242,  0.4241,  0.2875],
        ...,
        [ 0.1932, -0.4889,  0.1380,  ..., -1.0026,  0.1322, -0.7923],
        [-0.1220, -0.6803, -0.4610,  ..., -1.7360, -2.0502, -0.9297],
        [ 0.1274,  0.1923, -0.3626,  ...,  0.2435, -0.0061, -0.6971]],
       device='cuda:0')
Parameter containing:
tensor([[ 1.3103, -0.4103,  0.0046,  ..., -0.8584,  0.7383, -0.5054],
        [-0.0478,  1.4575, -1.2261,  ..., -0.8725,  1.6047,  1.0097],
        [-2.4463, -1.7865, -0.6676,  ..., -1.1606, -0.0412, -0.9194],
        ...,
        [ 1.4791,  0.7601,  1.6334,  ..., -1.4139, -1.5080,  2.5790],
        [ 1.3305, -1.0195, -1.3983,  ..., -0.4185,  2.0925,  0.0480],
        [ 1.5384,  0.2918,  1.2207,  ...,  2.0959,  1.4326, -4.5045]],
       device='cuda:0', requires_grad=True)
Parameter containing

In [36]:
print( model.fc1.weight )
print( model.fc1.parametrizations.weight[0].lora_A )
print( model.fc1.parametrizations.weight[0].lora_B )
print()
print( model.fc2.weight )
print( model.fc2.parametrizations.weight[0].lora_A )
print( model.fc2.parametrizations.weight[0].lora_B )
print()
print( model.fc3.weight )
print( model.fc3.parametrizations.weight[0].lora_A )
print( model.fc3.parametrizations.weight[0].lora_B )

tensor([[ 0.4101,  0.6101, -0.7784,  ...,  0.1910, -0.3317,  0.3116],
        [ 0.2397, -0.1602, -0.0614,  ..., -0.0139,  0.7553, -0.2833],
        [-0.2825, -0.0330, -0.0121,  ...,  0.6242,  0.4241,  0.2875],
        ...,
        [ 0.1932, -0.4889,  0.1380,  ..., -1.0026,  0.1322, -0.7923],
        [-0.1220, -0.6803, -0.4610,  ..., -1.7360, -2.0502, -0.9297],
        [ 0.1274,  0.1923, -0.3626,  ...,  0.2435, -0.0061, -0.6971]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[ 0.0045, -0.0224,  0.0418,  ..., -0.0270, -0.0161, -0.0293],
        [ 0.0445,  0.0337,  0.0454,  ...,  0.0097, -0.0300,  0.0135],
        [-0.0270,  0.0428, -0.0304,  ..., -0.0315,  0.0013,  0.0324],
        ...,
        [-0.0214,  0.0154, -0.0339,  ..., -0.0068, -0.0353,  0.0288],
        [-0.0398, -0.0296, -0.0346,  ..., -0.0406, -0.0293,  0.0298],
        [ 0.0302,  0.0198,  0.0044,  ...,  0.0285,  0.0111, -0.0377]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([[0., 0., 0.,  ..., 0., 0.

In [38]:
print( model.fc1.weight )
print( model.fc1.parametrizations.weight[0].lora_A )
print( model.fc1.parametrizations.weight[0].lora_B )
print()
print( model.fc2.weight )
print( model.fc2.parametrizations.weight[0].lora_A )
print( model.fc2.parametrizations.weight[0].lora_B )
print()
print( model.fc3.weight )
print( model.fc3.parametrizations.weight[0].lora_A )
print( model.fc3.parametrizations.weight[0].lora_B )

tensor([[ 0.0411,  0.6837, -0.7472,  ...,  0.1592, -0.2402, -0.0792],
        [-0.0900, -0.9739, -0.0067,  ...,  0.0168,  0.5582,  0.8106],
        [-0.1236, -0.2490, -0.0379,  ...,  0.6027,  0.9698,  0.4862],
        ...,
        [ 0.0659, -0.4628, -0.3113,  ..., -0.6657, -0.8887, -1.5158],
        [ 0.0145, -1.0332, -0.2172,  ..., -2.2406, -2.5094, -1.5576],
        [ 0.7025, -0.2122, -1.4907,  ..., -0.0513,  0.3789,  0.8024]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[ 0.0045, -0.0224,  0.0418,  ..., -0.0270, -0.0161, -0.0293],
        [ 0.0445,  0.0337,  0.0454,  ...,  0.0097, -0.0300,  0.0135],
        [-0.0270,  0.0428, -0.0304,  ..., -0.0315,  0.0013,  0.0324],
        ...,
        [-0.0214,  0.0154, -0.0339,  ..., -0.0068, -0.0353,  0.0288],
        [-0.0398, -0.0296, -0.0346,  ..., -0.0406, -0.0293,  0.0298],
        [ 0.0302,  0.0198,  0.0044,  ...,  0.0285,  0.0111, -0.0377]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([[0., 0., 0.,  ..., 0., 0.

In [25]:



# class LowRankParametrization(nn.Module):
#     def __init__(self, param, rank):
#         super().__init__()
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         self.rank = rank
#         self.u = nn.Parameter(torch.randn(param.size(0), rank)).to(device)
#         self.v = nn.Parameter(torch.randn(rank, param.size(1))).to(device)

#     def forward(self, X):
#         return X+torch.matmul(self.u, self.v)

# rank = 5  # Desired rank

# # Ensure the model's weight is initialized if needed
# parametrization1 = LowRankParametrization(model.fc1.weight, rank)
# parametrize.register_parametrization(model.fc1, 'weight', parametrization1)

# parametrization2 = LowRankParametrization(model.fc2.weight, rank)
# parametrize.register_parametrization(model.fc2, 'weight', parametrization2)

# parametrization3 = LowRankParametrization(model.fc3.weight, rank)
# parametrize.register_parametrization(model.fc3, 'weight', parametrization3)

optimizer_lora = torch.optim.Adam(model.parameters(), lr=0.01)

launch_experiment(model, device, train_loader, test_loader, optimizer_lora, 2, epoch_base)

Train Err	Test Err	Adv Err


  0%|          | 0/2 [00:00<?, ?it/s]

0.894483	0.897200	0.000000
0.897900	0.886500	0.000000


In [9]:
model.fc1.parametrizations #.weight[0].lora_A

AttributeError: 'Linear' object has no attribute 'parametrizations'

In [7]:
print( model.fc1.parametrizations.weight[0].lora_A )
print( model.fc1.parametrizations.weight[0].lora_B )

tensor([[ 1.2010e-02,  3.3419e-02,  2.3072e-03,  ..., -2.7030e-02,
          6.6422e-03, -8.2613e-03],
        [ 2.5634e-02, -1.2746e-02, -4.7297e-02,  ..., -3.3771e-02,
          1.5986e-05, -7.8924e-03],
        [ 2.5030e-02, -7.2367e-03, -2.7895e-03,  ...,  2.4114e-02,
         -1.2479e-02, -2.7373e-02],
        ...,
        [-5.6186e-03,  3.6513e-02,  1.7150e-03,  ...,  4.4266e-02,
         -3.6977e-02,  4.3982e-02],
        [ 1.0757e-03, -3.6546e-02, -1.8330e-02,  ..., -1.4210e-02,
         -2.2639e-02, -3.1511e-02],
        [ 2.6687e-02, -3.8629e-02,  1.7767e-02,  ...,  4.4685e-02,
          2.0076e-02, -2.1628e-03]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)


In [11]:
device = 'cuda'

model = LeNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

launch_experiment(model, device, train_loader, test_loader, optimizer, 10, epoch_AT_vanilla)

Train Err	Test Err	Adv Err


  0%|          | 0/10 [00:00<?, ?it/s]

0.091950	0.023900	0.034500
0.031017	0.012200	0.022500
0.022017	0.011000	0.019400
0.018900	0.011100	0.016300
0.016267	0.008800	0.017800
0.055883	0.041500	0.045100
0.036633	0.017700	0.031100
0.031867	0.015900	0.030600
0.030267	0.017800	0.028500
0.028067	0.016000	0.032700


In [12]:
device = 'cuda'

model = LeNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

launch_experiment(model, device, train_loader, test_loader, optimizer, 10, epoch_AT_free)

Train Err	Test Err	Adv Err


  0%|          | 0/10 [00:00<?, ?it/s]

0.103967	0.015100	0.023700
0.037150	0.011400	0.018900
0.027267	0.008600	0.015800
0.022833	0.009400	0.014500
0.019883	0.009700	0.014400
0.059267	0.022300	0.029700
0.037867	0.014800	0.024800
0.036583	0.015700	0.020700
0.034833	0.016500	0.023600
0.031733	0.013200	0.020600


In [ ]:
import torch
import torch.nn.functional as F

model.eval()
count = 0  # Initialize a counter
with torch.no_grad():  # No need to track gradients for testing
    for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            acc_ref = torch.zeros([1]).cuda()

            sign = 2 * ( (outputs.max(1)[1] == labels).float() ) - 1
            pred_var = (torch.var(F.softmax(outputs, dim=1), dim=1) * sign).detach()

            p = torch.clamp(1.0-acc_ref, 0, 0.99) # in [0, 1]

            th = F.relu(torch.quantile(pred_var, p))

            pred_robust = (pred_var > th).int()
            pred_boundary = torch.logical_and(pred_var <= th, pred_var >= 0).int()

            _, predicted = torch.max(outputs.data, 1)

            gamma = 0.8

            batch_acc = gamma * (logits_adv.max(1)[1] == y).detach().float().mean()

            acc_ref = 0.9 * acc_ref + 0.1 * batch_acc

            taille_r = sum(pred_robust)
            taille_b = sum(pred_boundary)

            count += 1  # Increment the counter for each batch
            # if count % 10 == 0:  # Check if the count is a multiple of 10
            print(f"Batch {count} -  Robust - {taille_r},  Boundary - {taille_b}")
